In [ ]:
import os
import xarray as xr
import numpy as np
import glob
# import geopandas as gp

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

import lmom as lmom
import MetFunctions02 as mf1
from scipy.stats import spearmanr
from scipy.stats import norm

from datetime import datetime

Open the 50000 AM peak files

Random selection according to N, pero set of AM

Calculate the metrics per confluence (100 runs pero sample)  and stores them according to the confluences


In [ ]:
# Random selection

In [ ]:
def randSet(am1, n):  #CHANGED --> now POT also uses the window as an imput
    """
    Creates DataFrame set of random Annual Maxima (AM) of one point
    AM1: DataFrame with the AM 
    n: size of the random set e.g. (50)
    Returns:
        set1: random set of AM of Point1 and concurrent flow of Point2
    """
    am1['ind'] = np.arange(len(am1))
    randomindex1 = np.random.choice(am1.ind, size=n, replace=False)
    set1 = pd.DataFrame()
    for i in range(n):
        set1 = set1.append(am1.loc[am1.ind == randomindex1[i]])
    return set1

def randSetR(am1, n):  #CHANGED --> now POT also uses the window as an imput
    """
    Creates DataFrame set of random Annual Maxima (AM) of one point
    AM1: DataFrame with the AM 
    n: size of the random set e.g. (50)
    Returns:
        set1: random set of AM of Point1 and concurrent flow of Point2
    """
    am1['ind'] = np.arange(len(am1))
    randomindex1 = np.random.choice(am1.ind, size=n, replace=True)
    set1 = pd.DataFrame()
    for i in range(n):
        set1 = set1.append(am1.loc[am1.ind == randomindex1[i]])
    return set1

In [ ]:
#Create the dataframes to be filled per N-sample

def dfN():
    conf_AM = pd.read_csv('Confluences_wnames.csv', delimiter=';')
    conf_AM['rho_mean'] = ''
    conf_AM['rho_std'] = ''
#     conf_AM['rho_cv'] = ''
    conf_AM['S1avg_mean'] = ''
    conf_AM['S1avg_std'] = ''
    conf_AM['S1std_mean'] = ''
    conf_AM['S1std_std'] = ''
#     conf_AM['S1avg_cv'] = ''
    conf_AM['S2avg_mean'] = ''
    conf_AM['S2avg_std'] = ''
    conf_AM['S2std_mean'] = ''
    conf_AM['S2std_std'] = ''
#     conf_AM['S2avg_cv'] = ''
    return conf_AM

In [ ]:
# Create ALL the dataframes per AM set, per N-sample size
# extra = ''
# for A in ('AM1', 'AM2', 'AM3'):
#     for n in ('100', '500', '1000', '5000', '10000', '25000', '50000'): #'100', 
#         globals()[f'C_{A}_n{n}{extra}'] = dfN()

# Create 1 w/extra info  N=100 r=50
extra = 'r5d'
n = 100 #100, 500, 1000, 5000, 10000, 25000, 50000
for A in ('AM1', 'AM2', 'AM3'): #'AM1', 
    globals()[f'C_{A}_n{n}{extra}'] = dfN()
    

In [ ]:
# Save ALL the dataframes
# desc = '' #SAVE'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
# extra = '' #RUN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
# for A in (1, 2, 3):
#     for n in (100, 500, 1000, 5000, 10000, 25000, 50000):
#         sv = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\METmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
#         globals()[f'C_AM{A}_n{n}{extra}'].to_pickle(sv)

# Save 1 dataframe
desc = 'check' #'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
extra = '' #RUN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
n = 1000 #100, 500, 1000, 5000, 10000, 25000, 50000
for A in (2, 3):
    sv = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AM-Nmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
#     sv = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\METmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
    globals()[f'C_AM{A}_n{n}{extra}'].to_pickle(sv)    

In [ ]:
# Open ALL the dataframes
# desc = '' #OPEN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
# extra = '' #RUN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
# for A in (1, 2, 3):
#     for n in (100, 500, 1000, 5000, 10000, 25000, 50000):
#         op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\METmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
#         globals()[f'C_AM{A}_n{n}{extra}'] = pd.read_pickle(op)

# Open 1 dataframe
desc = 'rain5d' #'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
extra = '5d' #RUN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
n = 100 #100, 500, 1000, 5000, 10000, 25000, 50000
for A in (1, 2, 3):
    op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\METmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
#     op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AM-Nmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
    globals()[f'C_AM{A}_n{n}{extra}'] = pd.read_pickle(op)

In [ ]:
#for running multiple times...   1st C0, N=100

def runsAM3(AM, n, runs):
    C_AM = pd.DataFrame(columns=['rho', 'S1avg_mean', 'S1avg_std', 'S2avg_mean', 'S2avg_std']) #, 'Savg_std', 'Savg_cv', 'gevF1_0', 'gevF1_1', 'gevF1_2', 'gevF2_0', 'gevF2_1', 'gevF2_2'])
    for i in range(runs): #100
        gev_fit1, gev_fit2, rho, SS1sav_mean, SS1sav_std,  SS2sav_mean, SS2sav_std = mf1.met(randSet(AM, n))
        C_AM.loc[(i,'rho')] = rho
        C_AM.loc[(i, 'S1avg_mean')] = SS1sav_mean
        C_AM.loc[(i, 'S2avg_mean')] = SS2sav_mean
        C_AM.loc[(i, 'S1avg_std')] = SS1sav_std
        C_AM.loc[(i, 'S2avg_std')] = SS2sav_std


    rho_mean = np.mean(C_AM.rho)
    rho_std = np.std(C_AM.rho)
#     rho_cv = rho_std / rho_mean
    #S1 mean & STD
    S1avg_mean = np.mean(C_AM.S1avg_mean)
    S1avg_std = np.std(C_AM.S1avg_mean)
#     S1avg_cv = S1avg_std / S1avg_mean
    S1std_mean = np.mean(C_AM.S1avg_std)
    S1std_std = np.std(C_AM.S1avg_std)
    #S2 mean & STD
    S2avg_mean = np.mean(C_AM.S2avg_mean)
    S2avg_std = np.std(C_AM.S2avg_mean)
#     S2avg_cv = S2avg_std / S2avg_mean
    S2std_mean = np.mean(C_AM.S2avg_std)
    S2std_std = np.std(C_AM.S2avg_std)
    
    return rho_mean, rho_std, S1avg_mean, S1avg_std, S1std_mean, S1std_std, S2avg_mean, S2avg_std, S2std_mean, S2std_std


def runsAM3R(AM, n, runs):
    C_AM = pd.DataFrame(columns=['rho', 'S1avg_mean', 'S1avg_std', 'S2avg_mean', 'S2avg_std']) #, 'Savg_std', 'Savg_cv', 'gevF1_0', 'gevF1_1', 'gevF1_2', 'gevF2_0', 'gevF2_1', 'gevF2_2'])
    for i in range(runs):
        gev_fit1, gev_fit2, rho, SS1sav_mean, SS1sav_std,  SS2sav_mean, SS2sav_std = mf1.met(randSetR(AM, n))
        C_AM.loc[(i,'rho')] = rho
        C_AM.loc[(i, 'S1avg_mean')] = SS1sav_mean
        C_AM.loc[(i, 'S2avg_mean')] = SS2sav_mean
        C_AM.loc[(i, 'S1avg_std')] = SS1sav_std
        C_AM.loc[(i, 'S2avg_std')] = SS2sav_std

    rho_mean = np.mean(C_AM.rho)
    rho_std = np.std(C_AM.rho)
#     rho_cv = rho_std / rho_mean
    #S1 mean & STD
    S1avg_mean = np.mean(C_AM.S1avg_mean)
    S1avg_std = np.std(C_AM.S1avg_mean)
#     S1avg_cv = S1avg_std / S1avg_mean
    S1std_mean = np.mean(C_AM.S1avg_std)
    S1std_std = np.std(C_AM.S1avg_std)
    #S2 mean & STD
    S2avg_mean = np.mean(C_AM.S2avg_mean)
    S2avg_std = np.std(C_AM.S2avg_mean)
#     S2avg_cv = S2avg_std / S2avg_mean
    S2std_mean = np.mean(C_AM.S2avg_std)
    S2std_std = np.std(C_AM.S2avg_std)
    
    return rho_mean, rho_std, S1avg_mean, S1avg_std, S1std_mean, S1std_std, S2avg_mean, S2avg_std, S2std_mean, S2std_std


In [ ]:
conf = pd.read_csv('Confluences_wnames.csv', delimiter=';')


In [ ]:
# FOR n = 100, 500, 1000, 5000, 10000
desc = 'rain' #'rain1d', 'snow', 'rain'
extra = 'r5d'
n = 50 #100, 500, 1000, 5000, 10000
runs = 100 # 100 (n=100, 500, 1000), 50 (n=1000, 5000, 10000)

for c in range(0, len(conf)): #len(conf)
    print(f'Starting {c} at {str(datetime.now().time())}')
    op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AMmet\C%d_" %c + desc + r"_" 
    AM1 = pd.read_pickle(op + "AM1.pkl")
    AM2 = pd.read_pickle(op + "AM2.pkl")
    AM3 = pd.read_pickle(op + "AM3.pkl")
    for A in ('AM1', 'AM2', 'AM3'): #'AM1', 
        rho_mean, rho_std, S1avg_mean, S1avg_std, S1std_mean, S1std_std, S2avg_mean, S2avg_std, S2std_mean, S2std_std = runsAM3(globals()[f'{A}'], n, runs)
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'rho_mean')] = rho_mean
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'rho_std')] = rho_std
#         globals()[f'C_{A}_n{n}'].loc[(c,'rho_cv')] = rho_cv
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1avg_mean')] = S1avg_mean
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1avg_std')] = S1avg_std
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1std_mean')] = S1std_mean
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1std_std')] = S1std_std
#         globals()[f'C_{A}_n{n}'].loc[(c,'S1avg_cv')] = S1avg_cv
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2avg_mean')] = S2avg_mean
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2avg_std')] = S2avg_std
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2std_mean')] = S2std_mean
        globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2std_std')] = S2std_std
#         globals()[f'C_{A}_n{n}'].loc[(c,'S2avg_cv')] = S2avg_cv
        print(f'Done C{A}')


In [ ]:
# Save 1 dataframe
desc = 'rain5d' #'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
extra = 'r5d' #RUN 'rain1d', 'snow', 'rain5d', (ONLY N=1000)'r50'
n = 50 #100, 500, 1000, 5000, 10000, 25000, 50000
for A in (1, 2, 3):
#     sv = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AM-Nmetrics\C_AM%d" %A + r"_n%d" %n  + desc + r".pkl"
    sv = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\METmetrics2\N%d" %n + r"_AM%d_" %A + desc + r".pkl"
    globals()[f'C_AM{A}_n{n}{extra}'].to_pickle(sv)    
    print(f'Saved {A}_n{n} at {str(datetime.now().time())}')

In [ ]:
# FOR n = 25000
desc = '' #'rain1d', 'snow', 'rain'
extra = ''
n = 25000
runs = 10

for c in range(0, len(conf)): #len(conf)
    print(f'Starting {c}')
    op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AMmet\C%d_" %c + desc + r"_" 
    AM1 = pd.read_pickle(op + "AM1.pkl")
    AM2 = pd.read_pickle(op + "AM2.pkl")
    AM3 = pd.read_pickle(op + "AM3.pkl")
    for A in ('AM1', 'AM2', 'AM3'):
        for a in (AM1, AM2, AM3):
            rho_mean, rho_std, S1avg_mean, S1avg_std, S1std_mean, S1std_std, S2avg_mean, S2avg_std, S2std_mean, S2std_std = runsAM3R(a, n, runs)
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'rho_mean')] = rho_mean
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'rho_std')] = rho_std
    #         globals()[f'C_{A}_n{n}'].loc[(c,'rho_cv')] = rho_cv
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1avg_mean')] = S1avg_mean
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1avg_std')] = S1avg_std
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1std_mean')] = S1std_mean
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1std_std')] = S1std_std
    #         globals()[f'C_{A}_n{n}'].loc[(c,'S1avg_cv')] = S1avg_cv
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2avg_mean')] = S2avg_mean
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2avg_std')] = S2avg_std
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2std_mean')] = S2std_mean
            globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2std_std')] = S2std_std
    #         globals()[f'C_{A}_n{n}'].loc[(c,'S2avg_cv')] = S2avg_cv
    print(f'Done C{c}')  

In [ ]:
# FOR n = 50000
desc = '' #'rain1d', 'snow', 'rain'
extra = ''
n = 50000
runs = 10
A = 'AM1' # 'AM1','AM2', 'AM3'


for c in range(0, len(conf)): #len(conf)
    n=50000 #Full data
    print(f'Starting {c}')
    op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AMmet\C%d_" %c + desc + r"_" 
    AM1 = pd.read_pickle(op + "AM1.pkl")
    AM2 = pd.read_pickle(op + "AM2.pkl")
    AM3 = pd.read_pickle(op + "AM3.pkl")

    gev_fit1, gev_fit2, rho, S1avg_mean, S1std_mean,  S2avg_mean, S2std_mean = mf1.met(AM3) #
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'rho_mean')] = rho
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'rho_std')] = 0
#         globals()[f'C_{A}_n{n}'].loc[(c,'rho_cv')] = rho_cv
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1avg_mean')] = S1avg_mean
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1avg_std')] = 0
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1std_mean')] = S1std_mean
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S1std_std')] = 0
#         globals()[f'C_{A}_n{n}'].loc[(c,'S1avg_cv')] = S1avg_cv
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2avg_mean')] = S2avg_mean
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2avg_std')] = 0
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2std_mean')] = S2std_mean
    globals()[f'C_{A}_n{n}{extra}'].loc[(c,'S2std_std')] = 0

        
        
